In [1]:
from dataset import CO2_synthetic_dataset 
import numpy as np
import torch

root_dir = '/Users/yufengyang/Library/CloudStorage/GoogleDrive-yang6956@umn.edu/My Drive/KGML/PyKGML/datasets/'
data_path = root_dir +  'CO2/'
output_path = root_dir + 'test_results/'

input_data = 'recotest_data_scaled_v4_100sample.sav'
sample_index_file = "traindataset_split_year_v1.sav"

pretrained_model = "recotest_v11_exp4.sav_step1"
output_model = "recotest_v11_exp4_sample.sav_step2"
synthetic_data = "sys_data2.sav"

dataset = CO2_synthetic_dataset(data_path, input_data, output_path, sample_index_file)
dataset.load_step2_data()

dataset.prepare_step2_data()

x_scaler = dataset.X_scaler
y1_scaler = dataset.Y1_scaler
y2_scaler = dataset.Y2_scaler

input_output_scalers = np.concatenate((x_scaler, y1_scaler), axis=0)
print(y1_scaler, y2_scaler)

input_features = ['RADN', 'TMAX_AIR', 'TDIF_AIR', 'HMAX_AIR', 'HDIF_AIR', 'WIND', 'PRECN', 'Crop_Type', 'GPP', 'Year', 'TBKDS', 'TSAND', 'TSILT', 
                  'TFC', 'TWP', 'TKSat', 'TSOC', 'TPH', 'TCEC']
output_features = ['Ra','Rh','NEE']

scalers_file = root_dir + 'processed_data/CO2/input{}_output{}_scalers.sav'.format(x_scaler.shape[0], y1_scaler.shape[0])

torch.save({
    'x_scaler': x_scaler,
    'y1_scaler': y1_scaler,
    'y2_scaler': y2_scaler,
    'input_features':input_features,
    'output_features': output_features}, scalers_file
    )

torch.Size([6570, 100, 19]) torch.Size([6570, 100, 3]) torch.Size([18, 100, 1])
['RADN', 'TMAX_AIR', 'TDIF_AIR', 'HMAX_AIR', 'HDIF_AIR', 'WIND', 'PRECN', 'Crop_Type', 'GPP', 'Year', 'TBKDS', 'TSAND', 'TSILT', 'TFC', 'TWP', 'TKSat', 'TSOC', 'TPH', 'TCEC']
[[-1.35840237  2.58333516]
 [-0.97471869  0.85668427]
 [-0.52353591  2.99425459]] [[220.9503479  103.95001984]]


In [ ]:

data_co2_path = root_dir +  'CO2/'
for i, filename in enumerate(['co2_pretrain_data.sav', 'co2_finetune_data.sav']):
    if i == 0:
        output_features = ['Ra','Rh', 'NEE']
        co2_y_scaler = y1_scaler
    else:
        output_features = ['NEE','RECO']
        reco_scaler = torch.from_numpy(np.array([np.mean([y1_scaler[0,0], y1_scaler[1,0]]), (y1_scaler[0,1]**2 + y1_scaler[1,1]**2)**0.5]))
        nee_scaler = torch.from_numpy(y1_scaler[2,:])
        co2_y_scaler = torch.stack((nee_scaler, reco_scaler), dim=0)

    file = torch.load(data_co2_path + filename)
    X_train, X_test, Y_train, Y_test, y_scaler, input_features = file['X_train'], file['X_test'], file['Y_train'], file['Y_test'], file['y_scaler'], file['input_features']
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape, y_scaler.shape)
    file_out = root_dir + 'processed_data/CO2/' + filename
    torch.save({'X_train': X_train,
            'X_test': X_test,
            'Y_train': Y_train,
            'Y_test': Y_test,
            'x_scaler': x_scaler,
            'y_scaler': co2_y_scaler,
            'input_features':input_features,
            'output_features': output_features}, file_out)

# data_n2o_path = root_dir +  'N2O/'
# x_scaler = torch.load(root_dir + 'processed_data/N2O/input16_output5_scalers.sav')
# for filename in ['n2o_pretrain_data.sav', 'n2o_finetune_augment_data.sav']:
#     file = torch.load(data_n2o_path + filename)
#     X_train, X_test, Y_train, Y_test, y_scaler, input_features, output_features = file['X_train'], file['X_test'], file['Y_train'], file['Y_test'], file['y_scaler'], file['input_features'], file['output_features']
#     print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape, y_scaler.shape)
#     file_out = root_dir + 'processed_data/N2O/' + filename
#     torch.save({'X_train': torch.tensor(X_train),
#             'X_test': torch.tensor(X_test),
#             'Y_train': torch.tensor(Y_train),
#             'Y_test': torch.tensor(Y_test),
#             'x_scaler': torch.tensor(x_scaler),
#             'y_scaler': torch.tensor(y_scaler),
#             'input_features': input_features,
#             'output_features': output_features}, file_out)

torch.Size([20, 6570, 19]) torch.Size([80, 6570, 19]) torch.Size([20, 6570, 3]) torch.Size([80, 6570, 3]) (3, 2)
['Ra', 'Rh', 'NEE']
torch.Size([1, 37230, 19]) torch.Size([1, 8030, 19]) torch.Size([1, 37230, 2]) torch.Size([1, 8030, 2]) (2, 2)
['NEE', 'RECO']


/var/folders/yv/lllpcq5d7g9cxbvg8bj3tg080000gn/T/ipykernel_42682/130213717.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  file = torch.load(data_co2_path + filename)


In [8]:
for filename in ['CO2/co2_pretrain_data.sav', 'CO2/co2_finetune_data.sav', 'N2O/n2o_pretrain_data.sav', 'N2O/n2o_finetune_augment_data.sav']:
    file = torch.load(root_dir + 'processed_data/' + filename)
    X_train, X_test, Y_train, Y_test, y_scaler, input_features, output_features = file['X_train'], file['X_test'], file['Y_train'], file['Y_test'], file['y_scaler'], file['input_features'], file['output_features']
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape, y_scaler.shape)
    print(file['output_features'])

/var/folders/yv/lllpcq5d7g9cxbvg8bj3tg080000gn/T/ipykernel_42682/967114454.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  file = torch.load(root_dir + 'processed_data/'

torch.Size([20, 6570, 19]) torch.Size([80, 6570, 19]) torch.Size([20, 6570, 3]) torch.Size([80, 6570, 3]) (3, 2)
['Ra', 'Rh', 'NEE']
torch.Size([1, 37230, 19]) torch.Size([1, 8030, 19]) torch.Size([1, 37230, 2]) torch.Size([1, 8030, 2]) torch.Size([2, 2])
['NEE', 'RECO']
torch.Size([1600, 6570, 16]) torch.Size([380, 6570, 16]) torch.Size([1600, 6570, 5]) torch.Size([380, 6570, 5]) torch.Size([5, 2])
['N2O_FLUX', 'CO2_FLUX', 'WTR_3', 'NH4_3', 'NO3_3']
torch.Size([5000, 366, 16]) torch.Size([1000, 366, 16]) torch.Size([5000, 366, 10]) torch.Size([1000, 366, 10]) torch.Size([5, 2])
['N2O_FLUX', 'CO2_FLUX', 'WTR_3', 'NH4_3', 'NO3_3']
